This notebook is an adpation of the *Questioning the Answers: LLMs enter the Boardroom, Using Gen AI Tools to Harness Alpha from Earnings Calls by S&P Global Market Intelligence's Quantitative Research & Solutions (QRS) Group research*.

# 1. Overview
Earnings calls play a pivotal role in shaping investor perceptions. The quality of communication between executives and analysts can significantly influence company performance. Efficient Communicators—executives who deliver proactive presentations, anticipate market queries, and provide clear, on-topic answers to analysts’ questions—consistently outperform their peers. Conversely, Total Redirectors—executives who are reactive, fail to address analysts’ key inquiries during presentations, and provide off-topic responses—significantly underperform.

Executives' ability to anticipate investor concerns and maintain a focused dialogue fosters confidence and strategic communication. In contrast, failing to provide clarity when analysts seek additional information can lead to misalignment and breakdowns in transparency. A long (short) portfolio of Efficient Communicators (Total Redirectors) generates +515bps of annualized alpha.

This notebook serves as the blueprint for the research detailed in Quantitative Research & Solutions’ recent publication, "Questioninig the Answers: LLM's enter the Boardroom." It analyse executive on-topicness and proactiveness using the analysts questions, executives answers and LLM answers. This research harness alpha using LLM tools, including vector embeddings, vector cosine similarity, and the LLM quesiton answering.

# 2. Datasets

In order to run this notebook you need access to the following datasets from the Snowflake Marketplace:

|Name|Description |
|----|----|
|[S&P Dow Jones Indices via Xpressfeed](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D9M/s-p-global-market-intelligence-s-p-dow-jones-indices-via-xpressfeed)|	The premium index benchmarking data from major index providers allows investors to track and simulate index performance, and are fully integrated and linked within the S&P Global Market Intelligence data ecosystem, so you can focus on your investment strategy. Premium index data available via Xpressfeed covers index returns (prices), as well as licensed index and their index constituent data all in one place.|
|[ S&P Capital IQ Financials](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2N/s-p-global-market-intelligence-s-p-capital-iq-financials)|	S&P Capital IQ Financials provides global standardized financial statement data for over 180,000 companies, including over 95,000 active and inactive public companies, and As Reported data for over 150,000 companies. S&P Capital IQ Standardized Financials allows you to extend the scope of your historical analysis and back-testing models with consistent data from all filings of a company's historical financial periods including press releases, original filings, and all restatements.|
|[Global Events](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D38/s-p-global-market-intelligence-global-events)|The Global Events dataset provides details on upcoming and past corporate events such as earnings calls, shareholder/analyst meetings, expected earnings release dates and more. With deep history back to 2003, clients can leverage this dataset to derive signals and support trading models across asset classes, trading styles and frequencies. This dataset also helps in research & analysis, risk management & compliance, and trade surveillance workflows.|
|[GICS®](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D3S/s-p-global-market-intelligence-gics®) |The GICS dataset includes global industry classifications for public companies, both current and historical. Leverage GICS, jointly developed by S&P Global and MSCI, for a complete, consistent set of global sector and industry definitions. GICS has become the standard widely recognized by market participants worldwide. The GICS methodology assigns each public company to a sub-industry and corresponding industry, industry group and sector, according to the definition of its principle business activity. The current GICS sectors include Energy, Materials, Industrials, Consumer Discretionary, Consumer Staples, Health Care, Financials, Information Technology, Communication Services, Utilities, and Real Estate.|
|[Compustat® Financials](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2R/s-p-global-market-intelligence-compustat®-financials)	|Compustat Financials provides standardized North American and global financial statements and market data for over 80,000 active and inactive publicly traded companies that financial professionals have relied on for over 50 years. Compustat allows investment professionals, academic researchers, and industry analysts to combine deep history with robust and consistent data standardization into their research and backtesting to produce valuable insights and generate alpha. With historical data for North America as far back as 1950 and point-in-time snapshots beginning in 1987, Compustat provides you with insight into company financial performance across many different economic cycles not available anywhere else.|

# 3. Libraries & User Inputs
Import libraries required for workflow

## 3.1 Libraries

Need to add **cachetools** through **packages**

In [ ]:
# Import python packages
import streamlit as st
#import pandas as pd
from datetime import datetime
import cachetools

from snowflake.snowpark import functions as snow_funcs
from snowflake.snowpark import Window
from snowflake.snowpark.types import ArrayType, StringType

from snowflake.snowpark.context import get_active_session
session = get_active_session()

## 3.2 User Inputs

This research invloves the usage of an embedding model and a completion model, the default models were set to "snowflake-arctic-embed-m" for embedding and "llama3.1-8b" for completion. This user input section gives you the flexibility to chose your own model for the task.

In [ ]:
snf_embed_text_func = "SNOWFLAKE.CORTEX.EMBED_TEXT_768" # Which of the two embedding functions we want to use

embedding_model = "snowflake-arctic-embed-m"
completion_model = "llama3.1-8b"

# Name of the databse created in the Setup Snowflake step
sp_llm_qs_location = "SP_LLM_QS.PUBLIC"

# Name of the shared database craeted at the "Request the S&P Global Market Intelligence QuickStart dataset" step
sp_qs_share_location = "XF_SNOWFLAKE_QUICKSTRAT.XPRESSFEED"

## 3.3 Date Dimension

In [ ]:

beginDate = '2000-01-01' #start of period for analysis
endDate = '2023-12-31' #end of period for analysis

nbr_of_days = (datetime.strptime(endDate, "%Y-%m-%d") - datetime.strptime(beginDate, "%Y-%m-%d")).days


calendar_df = (session.generator(snow_funcs.to_date(snow_funcs.dateadd("DAY", snow_funcs.call_function("SEQ4"), snow_funcs.lit(beginDate))).as_("Date"), rowcount=nbr_of_days)
               .with_columns(["Year", "Month", "Day", "Quarter"],
                                  [snow_funcs.year(snow_funcs.col("Date"))
                                   , snow_funcs.month(snow_funcs.col("Date"))
                                   , snow_funcs.dayofyear(snow_funcs.col("Date"))
                                   , snow_funcs.quarter(snow_funcs.col("Date"))
                                   , 
                                  ])
               .with_columns(['FirstDayOfQuarter', 'LastDayOfQuarter']
                            , [snow_funcs.min(snow_funcs.col('Date')).over(Window.partition_by(snow_funcs.col('year'), snow_funcs.col('quarter')))
                              ,snow_funcs.last_day(snow_funcs.col('Date'), "QUARTER")])
              )

calendar_df.create_or_replace_temp_view(f"{sp_llm_qs_location}.date_dimension")
calendar_df.show()

# 4. Working with the Data: Russell 3000 Transcripts
In this section we query all available Russell 3000 constituent transcripts.

## 4.1 Index Data
### 4.1.1 Russell 3000 Constituents
In the cells below we set the date range for the analysis. The Russell constituents list is collected and formatted to show the daily constituents, including their date, GICS sector and calendarYearQuarter.

In [ ]:
df_indexcon = session.sql(f"""
select distinct
c.companyId, c.companyName
, s.securityId
, ic.tradingItemId, i.indexId, i.indexName, ic.constituentId
, ti.tickerSymbol, e.exchangeSymbol as exchangeSymbol
, concat(e.exchangeSymbol, ' : ', ti.tickerSymbol) as exchangeTicker
, to_date(ic.fromDate) as fromDate, to_date(ifnull(ic.toDate,current_date())) as toDate

from {sp_qs_share_location}.ciqIndex i
    join {sp_qs_share_location}.ciqindexConstituent ic on ic.indexId=i.indexId
    join {sp_qs_share_location}.ciqTradingItem ti on ti.tradingItemId=ic.tradingItemId
    join {sp_qs_share_location}.ciqExchange e on e.exchangeId=ti.exchangeId
    join {sp_qs_share_location}.ciqSecurity s on s.securityId=ti.securityId
    join (select companyId, companyName from {sp_qs_share_location}.ciqCompany c) c on c.companyId=s.companyId
where i.indexId in (
--2668699 --S&P 500
--2668861 --S&P 100
2668795 --Russell 3000
) 

""")

df_indexcon.write.save_as_table(f"{sp_llm_qs_location}.R3000HISTORYTOPRESENT", mode="overwrite", table_type='transient')
df_indexcon.limit(5)

### 4.1.2 Enrich Russell 3000 Constituents with sector and calendar year quarter

In [ ]:
df_indexcon = session.sql(f"""
select 
    dd.firstDayOfQuarter, dd.lastDayOfQuarter,
    concat(dd.year, '-', dd.quarter) as calendarYearQuarter,
    concat(cp.fiscalYear, '-', cp.fiscalQuarter) as fiscalYearQuarter,
    cpt.periodTypeName, cpt.periodTypeDescription,
    r.companyId, r.companyName, r.tradingItemId, gs.gicDesc as Sector, gg.gicDesc as SectorGroup, gind.gicDesc as Industry, gsi.gicDesc as subIndustry
from (select * from {sp_llm_qs_location}.DATE_DIMENSION where date = firstDayOfQuarter) dd
    join {sp_llm_qs_location}.R3000HISTORYTOPRESENT r on dd.date between ifnull(r.fromDate, '1950-01-01') and ifnull(r.toDate, '2050-12-31')
    join {sp_qs_share_location}.ciqFinPeriod cp on r.companyId = cp.companyId and dd.year = cp.calendarYear and dd.quarter = cp.calendarQuarter
    join {sp_qs_share_location}.ciqEventPeriodType cpt on cp.periodTypeId = cpt.periodTypeId
    
    left join {sp_qs_share_location}.ciqGvkeyIID gi on gi.RELATEDCOMPANYID=r.companyId and gi.objectId=r.tradingItemId and dd.FirstDayOfQuarter between ifnull(gi.symbolStartDate,'1950-01-01') and ifnull(gi.symbolEndDate,'2050-01-01')
    left join {sp_qs_share_location}.gic_history gic on gic.GVKEY=gi.GVKEY and dd.FirstDayOfQuarter between ifnull(gic.indFrom,'1950-01-01') and ifnull(gic.indthru,'2050-01-01')
    left join {sp_qs_share_location}.r_giccd gs on gs.giccd=gic.gsector
    left join {sp_qs_share_location}.r_giccd gg on gg.giccd=gic.ggroup
    left join {sp_qs_share_location}.r_giccd gind on gind.giccd=gic.gind
    left join {sp_qs_share_location}.r_giccd gsi on gsi.giccd=gic.gsubind

where 1=1
    and date between '2000-01-01' and '2024-12-31'
    and cp.periodTypeId = 2 --Fiscal/Calendar Quarter
order by calendarYearQuarter asc, companyId asc

""")

df_indexcon.write.save_as_table(f"{sp_llm_qs_location}.R3000HISTORYTOPRESENT_SECTORS", mode="overwrite", table_type='transient')
df_indexcon.limit(5)

## 4.2 Machine Readable Transcripts
### 4.2.1 Collect TranscriptId

In [ ]:

df_transcript = session.sql(f"""
                        
select * from 
(select 
    ice.*, t.transcriptId, row_number() over (partition by fiscalYearQuarter, tradingItemId, ice.keyDevId order by t.transcriptCreationDateUTC desc) as latestTranscriptforKeyDev_Flag
    
from
(select 
    ice.calendarYearQuarter, ice.fiscalYearQuarter, ice.tradingItemId, ice.companyId, ice.companyName, eot.keyDevId, e.headline
    , e.mostImportantDateUTC, enteredDate, bi.periodEndDate, languageId, keydeveventtypeid
    , row_number() over (partition by ice.tradingItemId, bi.fiscalYear, bi.fiscalQuarter order by e.lastModifiedDate desc) as latestKeyDev_Flag 
   from {sp_llm_qs_location}.R3000HISTORYTOPRESENT_SECTORS ice 
    join {sp_qs_share_location}.ciqEventToObjectToEventType eot on ice.companyId = eot.objectId and eot.keydeveventtypeid=48
    join {sp_qs_share_location}.ciqEvent e on eot.keydevid = e.keydevid
    join {sp_qs_share_location}.ciqEventCallBasicInfo bi on bi.keyDevId=e.KeyDevId and concat(bi.fiscalYear,'-',bi.fiscalQuarter) = ice.fiscalYearQuarter 
    
where bi.languageId in (0, 123) OR bi.languageId IS NULL --English
) ice
    left join {sp_qs_share_location}.ciqTranscript t on t.keyDevId=ice.keyDevId
where 1=1 
    and t.transcriptId is not null
    and latestKeyDev_Flag = 1)
where latestTranscriptforKeyDev_Flag = 1 
order by calendarYearQuarter asc, tradingItemId asc

""")


df_transcript.write.save_as_table(f"{sp_llm_qs_location}.R3000HISTORYTOPRESENT_Transcript", mode="overwrite", table_type='transient')
df_transcript.limit(5)

### **4.2.2 Collect TranscriptId Components
This query takes about 30 minutes to executive since it is collecting all Russell 3000 transcript components in the history.

In [ ]:

df_transcriptComponents = session.sql(f"""
select distinct 
    u.*
    , speakerTypeName
    , tp.transcriptPersonName
    , tp.transcriptPersonId
    , tp.proId
    , ct.transcriptComponentTypeId
    , transcriptComponentTypeName
    , componentOrder
    , transcriptComponentId
    , componentText

from
(select distinct to_date(mostImportantDateUTC) as callDate, to_date(enteredDate) as enteredDate, fiscalYearQuarter
         , calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId 
         from {sp_llm_qs_location}.R3000HISTORYTOPRESENT_Transcript) u   
    join {sp_qs_share_location}.ciqTranscriptComponent tc on tc.transcriptId=u.transcriptId
    join {sp_qs_share_location}.ciqTranscriptComponentType ct on ct.transcriptComponentTypeId=tc.transcriptComponentTypeId
    join {sp_qs_share_location}.ciqTranscriptPerson tp on tp.transcriptPersonId=tc.transcriptPersonId
    join {sp_qs_share_location}.ciqTranscriptSpeakerType st on st.speakerTypeId=tp.speakerTypeId
order by companyId asc, callDate asc, transcriptId asc, componentOrder asc

""")

df_transcriptComponents.write.save_as_table(f"{sp_llm_qs_location}.R3000HISTORYTOPRESENT_TranscriptComponents", mode="overwrite", table_type='transient')
df_transcriptComponents.limit(5)

### 4.2.3 Coverage Check

In [ ]:
select calendarYearQuarter, count(distinct transcriptId) 
from {{sp_llm_qs_location}}.R3000HISTORYTOPRESENT_TRANSCRIPTCOMPONENTS 
group by calendarYearQuarter 
order by calendarYearQuarter

# 5. Working with the Data: Sentence Tokenization
In this section we tokenize the prepared remark and the answer components on the sentence level.

## 5.1 Define Sentence Tokenization Function

In order to run the this step and the following you need to make sure you have created the  NETWORK RULE and  EXTERNAL ACCESS INTEGRATION objects in the **Create Database, Schema And Warehouse To Be Used** step of the quickstart guide.

Create a UDF to do the tokenizing.

In [ ]:
# Using @cachetools.cached ensures that we only download them once
@cachetools.cached(cache={})
def download_files(save_path):
    import nltk
    import os
    
    os.environ['NLTK_DATA'] = save_path

    # Create the directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)

    nltk.download('punkt', download_dir=save_path)
    nltk.download('punkt_tab', download_dir=save_path) 
                
def sentenceTokenize(x):
    import nltk
    from nltk.tokenize import sent_tokenize

    # Set the NLTK_DATA environment variable to the desired directory
    nltk_data_dir = '/tmp/nltk_data'
    nltk.data.path.append(nltk_data_dir)
    download_files(nltk_data_dir)
    
    return sent_tokenize(x)

sentence_tokenize_udf = snow_funcs.udf(sentenceTokenize
                                       , return_type = ArrayType(StringType())
                                       , input_types=[StringType()]
                                      , packages=['nltk', 'cachetools']
                                      , external_access_integrations=['NLTK_ACCESS_INTEGRATION'])

## 5.2 Apply Tokenize Sentence Function
The code below takes about X minutes to run, since it is sentence tokenizing all historical prepared remarks.



In [ ]:
df = session.table(f"{sp_llm_qs_location}.R3000HISTORYTOPRESENT_TRANSCRIPTCOMPONENTS")

filtered_df = df.filter((snow_funcs.col('transcriptComponentTypeId') == 2) | (snow_funcs.col('transcriptComponentTypeId') == 4))

# Apply the UDF to the DataFrame column
df_with_sentences = filtered_df.withColumn("sentencesList", sentence_tokenize_udf(filtered_df["componentText"])).cache_result()

# Explode the sentencesList column and generate the order column
df_exploded = (df_with_sentences.select( *[col for col in df_with_sentences.columns if col != "sentencesList"]
                          ,snow_funcs.flatten(snow_funcs.col("sentencesList")).alias("SEQ", "KEY", "PATH"
                                                                                     ,  "sentenceIndex", "sentence", "THIS"))
                .drop("SEQ", "KEY", "PATH", "THIS")
                .with_column("sentence",snow_funcs.as_char(snow_funcs.col("sentence")))
              )

df_exploded.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_PPP_A_Sentences", mode="overwrite", table_type='transient')

df_exploded.limit(5)

# 6. Working with the Data: Retrival Augmented Generation (RAG)
Retrieval-Augmented Generation (RAG) is a tool that improves LLM consistency by retrieving relevant information before answering a question.

For this task the LLM needs to be as consistent as possible in its responses to the analysts’ questions as inconsistency will lead to variations in cosine similarity scores and disrupt feature generation downstream.

To combat this, we designed a Retrieval-Augmented Generation (RAG) engine that chunks the prepared remarks sentence-by-sentence and retrieves the optimal retrieval percentage of sentences most similar to the question. Inconsistency occurs when the LLM is provided with too little (or too much) context, it becomes uninformed (unspecific). The optimal retrieval percentage for consistency is 60%.

In the cells below, we vector embed all questions, prepared remark sentences and answer sentences using the native databricks-gte-large-en embedding model. Then use the cosine similarity from the (question vs prepared remark sentences) and (question vs answer sentences) to select top 60% most relevant prepared remark and answer sentences to the question from S&P Global Q3 2024 Earnings Call Transcript

## 6.1 S&P Global Q3 2024 Earnings Call Transcript
Select and seperate the prepared remarks, questions and answers sections of the S&P Global Q3 2024 Earnings Call

In [ ]:
sentences_df = session.table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_PPP_A_Sentences")
all_component_df = session.table(f"{sp_llm_qs_location}.R3000HISTORYTOPRESENT_TRANSCRIPTCOMPONENTS")
pppSentences = sentences_df.filter(snow_funcs.col('transcriptComponentTypeId') == 2)
questions = all_component_df.filter(snow_funcs.col('transcriptComponentTypeId') == 3)
answerSentences = sentences_df.filter(snow_funcs.col('transcriptComponentTypeId') == 4)

st.dataframe(pppSentences.limit(5))
st.dataframe(questions.limit(5))
st.dataframe(answerSentences.limit(5))

## 6.2 Vector Embedding


### 6.2.2 Apply Embedding to Transcript Components

In [ ]:
snf_embed_text = snow_funcs.function(snf_embed_text_func)

pppSentences_with_embeddings = pppSentences.withColumn("sentenceVec", snf_embed_text(snow_funcs.lit(embedding_model)
                                                                                     ,pppSentences["sentence"]))
questions_with_embeddings = questions.withColumn("componentTextVec", snf_embed_text(snow_funcs.lit(embedding_model),
                                                                                    questions["componentText"]))
answerSentences_with_embeddings = answerSentences.withColumn("sentenceVec",  snf_embed_text(snow_funcs.lit(embedding_model)
                                                                                            ,answerSentences["sentence"]))

pppSentences_with_embeddings.write.save_as_table(f"{sp_llm_qs_location}.pppSentencesVec", mode="overwrite",table_type='transient')
questions_with_embeddings.write.save_as_table(f"{sp_llm_qs_location}.questionsVec", mode="overwrite", table_type='transient')
answerSentences_with_embeddings.write.save_as_table(f"{sp_llm_qs_location}.answerSentencesVec", mode="overwrite", table_type='transient')

# 6.3 Cosine Similarity
To determine semantic closeness, we vector-embed the question-and-answer texts and calculate a cosine similarity score between the two vectors.
For example, A and B each represent a vector such that:


$$	A = [a_1,a_2,… a_n] $$
$$	B = [b_1,b_2,… b_n] $$


The cosine similarity formula between vectors A and B is:

$$\text{Cosine Similarity} = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \cdot \|\mathbf{B}\|}$$

, where A⋅B is the dot product of the vectors, and |A|⋅|B| is the product of each vector's magnitude.
The result ranges from -1 to 1, where:

	-1: Vectors are opposite.
	0: Vectors are unrelated.
	1: Vectors are identical.

### 6.3.2 Apply Cosine Similarity to Question & Answer Vector Embeddings
The cell below creates question and answer pair by collecting all the answer sentences whose componentOrder is between the currentQuestionComponentOrder and the nextQuestionComponentOrder. Then the the consine similarity was applied to the question and answer sentence vector embeddings.

In [ ]:
df = session.sql(f"""

with q as(
    select 
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter,
        tradingItemId, companyId, companyName, headline, transcriptId,
        speakerTypeName as questionSpeakerTypeName, transcriptPersonName as questionTranscriptPersonName,
        transcriptPersonId as questionTranscriptPersonId, proId as questionProId, transcriptComponentTypeId as questionTranscriptComponentTypeId,
        transcriptComponentId as questionTranscriptComponentId, componentOrder as currentQuestionOrder
        , lead(componentOrder) over (partition by tradingItemId, transcriptId order by componentOrder) as nextQuestionOrder
        , componentText as question, componentTextVec as questionVec
    from {sp_llm_qs_location}.questionsVec
    where 1=1
        and transcriptComponentTypeId = 3
)
select 
    q.callDate, q.enteredDate, q.fiscalYearQuarter, q.calendarYearQuarter,
    q.tradingItemId, q.companyId, q.companyName, q.headline, q.transcriptId,
    q.questionSpeakerTypeName, q.questionTranscriptPersonName, q.questionTranscriptPersonId, q.questionProId, 
    a.speakerTypeName as answerSpeakerTypeName, a.transcriptPersonName as answerTranscriptPersonName, a.transcriptPersonId as answerTranscriptPersonId, a.proId as answerProId, 
    q.questionTranscriptComponentTypeId, a.transcriptComponentTypeId as answerTranscriptComponentTypeId,
    q.questionTranscriptComponentId, a.transcriptComponentId as answerTranscriptComponentId,
    q.currentQuestionOrder, q.nextQuestionOrder, a.componentOrder as answerOrder, a.sentenceIndex as answerSentenceOrder,
    q.question, a.componentText as answer, a.sentence as answerSentence, q.questionVec, a.sentenceVec as answerSentenceVec
    , VECTOR_COSINE_SIMILARITY(questionVec, sentenceVec) AS cosine_similarity
from q
    join {sp_llm_qs_location}.answerSentencesVec a on q.tradingItemId = a.tradingItemId and q.transcriptId = a.transcriptId and a.componentOrder between q.currentQuestionOrder and ifnull(q.nextQuestionOrder, 10000)
order by q.nextQuestionOrder asc, a.componentOrder asc, a.sentenceIndex asc

""")

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_qaPairCos", mode="overwrite", table_type='transient')
df.limit(5)

### 6.3.3 Apply Cosine Similarity to Question & Prepared Remarks Vector Embeddings
The cell below pairs all prepared remarks sentences to questions. Then the the consine similarity was applied to the question and prepared remarks sentence vector embeddings.

In [ ]:
df = session.sql(f"""

with q as(
    select 
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter,
        tradingItemId, companyId, companyName, headline, transcriptId,
        speakerTypeName as questionSpeakerTypeName, transcriptPersonName as questionTranscriptPersonName,
        transcriptPersonId as questionTranscriptPersonId, proId as questionProId, transcriptComponentTypeId as questionTranscriptComponentTypeId,
        transcriptComponentId as questionTranscriptComponentId, componentOrder as currentQuestionOrder, lead(componentOrder) over (partition by tradingItemId, transcriptId order by componentOrder) as nextQuestionOrder, componentText as question, componentTextVec as questionVec
    from {sp_llm_qs_location}.questionsVec
    where 1=1
        and transcriptComponentTypeId = 3
)
select 
    q.*, 
    p.componentOrder as executiveRemarkComponentOrder, p.sentenceIndex as executiveRemarkSentenceOrder,
    p.componentText as executiveRemark, p.sentence as executiveSentence, p.sentenceVec as executiveVec,
    VECTOR_COSINE_SIMILARITY(questionVec, p.sentenceVec) as similarity
from q
    join {sp_llm_qs_location}.pppSentencesVec p on q.tradingItemId = p.tradingItemId and q.transcriptId = p.transcriptId

order by q.nextQuestionOrder asc, p.componentOrder asc, p.sentenceIndex asc

""")

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairCos", mode="overwrite", table_type='transient')
df.limit(5)

## 6.4 Top 60% Sentences
Utilizing the top 60% of prepared remarks identified as generating the most consistent LLM output. For further details on the experiment, please refer to the 'LLM Robustness Check' section in the whitepaper.

### 6.4.2 Concat Top 60% Answer Sentences
After selecting the top 60% most similar answer sentences, we concat the answer sentences on the question level.

In [ ]:

df = session.sql(f'''
          
    with qaSimilarityRank as(
    select 
        *, 
        ROW_NUMBER() OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder, currentQuestionOrder ORDER BY cosine_similarity desc) AS similarityRank,
        COUNT(*) OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder, currentQuestionOrder) AS answerSentencesCount 
    from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_qaPairCos 
    ), 
    questionWithSixtyPercentAnswers as(

    select 
        *, 
        case 
            when answerSentencesCount = 1 then 1
            when similarityRank <= answerSentencesCount * 0.67 then 1
            else 0
        end as toKeepFlag
    from qaSimilarityRank

    ),
    qaPair60 as(
    select
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName
        , headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId
        , questionProId, answerSpeakerTypeName, answerTranscriptPersonName, answerTranscriptPersonId, answerProId
        , questionTranscriptComponentTypeId, answerTranscriptComponentTypeId, questionTranscriptComponentId
        , answerTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, answerOrder, question, answer, 
        ARRAY_TO_STRING(ARRAY_AGG(answerSentence), ' ') AS sixtyPercentAnswer
    from (select * from questionWithSixtyPercentAnswers where toKeepFlag = 1 order by currentQuestionOrder asc, answerOrder asc, answerSentenceOrder asc)
    group by callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, answerSpeakerTypeName, answerTranscriptPersonName, answerTranscriptPersonId, answerProId, questionTranscriptComponentTypeId, answerTranscriptComponentTypeId, questionTranscriptComponentId, answerTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, answerOrder, question, answer)
    select 
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, answerSpeakerTypeName, answerTranscriptPersonName, answerTranscriptPersonId, answerProId, questionTranscriptComponentTypeId, answerTranscriptComponentTypeId, questionTranscriptComponentId, answerTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, answerOrder, 
        question, answer
        , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',answer) as answerTokenCount
        , sixtyPercentAnswer
        , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',sixtyPercentAnswer) as sixtyPercentAnswerTokenCount
    from qaPair60
    order by currentQuestionOrder asc, answerOrder asc
              
''')

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_qaPairTop60", mode="overwrite", table_type='transient')
df.limit(5)

### 6.4.3 Concat Top 60% Prepared Remarks Sentences
Similarly, after selecting the top 60% most similar prepared remarks sentences, we concat the prepared remarks sentences on the question level.

In [ ]:

df = session.sql(f'''

with qaSimilarityRank as(
select 
    *, 
    ROW_NUMBER() OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder ORDER BY similarity desc) AS similarityRank,
    COUNT(*) OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder) AS executiveRemarksSentencesCount
from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairCos
),

pppQPairTop60 as(
select 
    *, 
    case 
        when executiveRemarksSentencesCount = 1 then 1
        when similarityRank <= executiveRemarksSentencesCount * 0.67 then 1
        else 0
    end as toKeepFlag
from qaSimilarityRank),

presenterLevelSixtyPercentPPP as(
select
    callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, executiveRemarkComponentOrder, question, executiveRemark, 
    ARRAY_TO_STRING(ARRAY_AGG(executiveSentence), ' ') AS sixtyPercentExecutiveRemark
from (select * from pppQPairTop60 where toKeepFlag = 1 order by currentQuestionOrder asc, executiveRemarkComponentOrder asc, executiveRemarkSentenceOrder asc)
group by callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, executiveRemarkComponentOrder, question, executiveRemark
order by currentQuestionOrder asc,  executiveRemarkComponentOrder asc),

pppQPairTop60Concat as(
select 

    callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, question,
    ARRAY_TO_STRING(ARRAY_AGG(executiveRemark), '\n\n') AS questionLevelConcatenatedRawExecutiveRemark,
    ARRAY_TO_STRING(ARRAY_AGG(sixtyPercentExecutiveRemark), '\n\n') AS questionLevelConcatenatedSixtyPercentExecutiveRemark
from (select * from presenterLevelSixtyPercentPPP order by currentQuestionOrder asc,  executiveRemarkComponentOrder asc)
group by callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, question
)

select 
    callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, question,
    questionLevelConcatenatedRawExecutiveRemark
    , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',questionLevelConcatenatedRawExecutiveRemark) as questionLevelConcatenatedRawExecutiveRemarkTokenCount, questionLevelConcatenatedSixtyPercentExecutiveRemark
    , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',questionLevelConcatenatedSixtyPercentExecutiveRemark) as questionLevelConcatenatedSixtyPercentExecutiveRemarkTokenCount
from pppQPairTop60Concat
order by currentQuestionOrder asc

''')

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairTop60", mode="overwrite", table_type='transient')
df.limit(5)

# 7. Working with the Data: LLM Ready Data
## 7.1 Using Snowflake Cortex AI
In the Snowflake Cortex AI COMPLETE finction, messages are organized into distinct roles—system, user, and assistant—to structure and guide interactions. Each role serves a specific purpose:

System: Provides instructions that define the context or behavior of the model. It's like setting the rules or tone for the conversation. Example: "You are a helpful assistant that answers questions about technology in a concise manner."

User: Represents the input or queries made by the person interacting with the model. These are the prompts or requests that the model responds to. Example: "What is the purpose of the OpenAI API?"

Assistant: Reflects the model's response to the user's query, shaped by the system's instructions and the user's input. Example: "The OpenAI API is designed to enable developers to integrate language models into their applications for tasks like answering questions, generating content, and more."

In our research, executive prepared remarks are labelled as assistant messages, analyst's questions as User messages and executive answers as Assistant messages

### 7.1.1 Construct COMPLETE Assistant Message with QA Pair Snippet

In [ ]:
df = session.sql(f'''

with questionWithSixtyPercentAnswersPromptSnippet as (

select
    *, 
    OBJECT_CONSTRUCT('role', 'user', 'content', REPLACE(question, '\r', '')) as questionPromptSnippet,
    OBJECT_CONSTRUCT('role', 'assistant', 'content', REPLACE(sixtyPercentAnswer, '\r', '')) as answerPromptSnippet
    
from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_qaPairTop60)
select 
    *, ARRAY_CONSTRUCT(questionPromptSnippet, answerPromptSnippet) as questionAnswerPairPromptSnippet 
from questionWithSixtyPercentAnswersPromptSnippet
order by currentQuestionOrder asc, answerOrder asc

''')


df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairTop60AnswerConcat", mode="overwrite", table_type='transient')
df.limit(5)


### 7.1.2 Construct OpenAI Assistant Message with Prepared Remarks Snippets

In [ ]:
df = session.sql(f'''

select
    *,
    OBJECT_CONSTRUCT('role', 'user', 'content', REPLACE(question, '\r', '')) as questionPromptSnippet,
    OBJECT_CONSTRUCT('role', 'assistant', 'content', REPLACE(questionLevelConcatenatedSixtyPercentExecutiveRemark, '\r', '')) as pppPromptSnippet
from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairTop60

''')

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairTop60PPPConcat", mode="overwrite", table_type='transient')
df.limit(5)

## 7.2 Collect All Messages for OpenAI API
All question pairs with prepare remarks and answers come together to form an LLM prompt following the iterative process such that:

1. 'user': 'From the perspective of a top executive, please answer the following question raised by a financial analyst during an earnings conference call. Knowledge cutoff date: '  
2. 'assistant': 60% prepared remarks  
3. 'user': question 1  
4. 'assistant': 60% answer 1  
5. ...  
6. ...  
7. 'user': question n  


### 7.2.1 LLM Ready Prompt Messages
In the dataframe below, the prompt column has all the messages in 1 list. This is the prompt for the LLM.

In [ ]:
df = session.sql(f'''
    with pppQPairTop60PPPConcat
    as
    (
    select 
         a.callDate, a.tradingItemId, a.transcriptId, a.headline, 
        a.questionTranscriptPersonName, a.questionTranscriptPersonId, a.questionProId,
        a.answerTranscriptPersonName, a.answerTranscriptPersonId, a.answerProId,
        a.questionTranscriptComponentId, a.answerTranscriptComponentId,
        a.question, a.answer,
        ARRAY_CONSTRUCT(object_construct('role', 'user', 'content', concat('From the perspective of a top executive, please answer the following question raised by a financial analyst during an earnings conference call. Knowledge cutoff date: ', cast(a.callDate as string)))) as initPrompt,
       a.questionPromptSnippet,
       ARRAY_FLATTEN(ARRAY_AGG(a.questionAnswerPairPromptSnippet) OVER (PARTITION BY a.TRANSCRIPTID ORDER BY a.CURRENTQUESTIONORDER 
                                                        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)) as concatenatedPredecessors
     from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairTop60AnswerConcat a
    )
    select a.* 
        , ARRAY_CAT(a.initPrompt, ARRAY_CAT([ppp.pppPromptSnippet], ARRAY_CAT(a.concatenatedPredecessors, [a.questionPromptSnippet]))) as prompt
    from 
    pppQPairTop60PPPConcat a
    join {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_pppQPairTop60PPPConcat as ppp on a.tradingItemId = ppp.tradingItemId and a.transcriptId = ppp.transcriptId and a.questionTranscriptComponentId = ppp.questionTranscriptComponentId
    order by a.questionTranscriptComponentId asc, a.answerTranscriptComponentId asc
''')

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_targetLLMReadyPrompt", mode="overwrite", table_type='transient')
df.limit(5)


## 7.3 Collect LLM Response


### 7.3.2 Apply LLM Completion
We apply the SNOWFLAKE.CORTEX.COMPLETE function on the prompt column using the model defined by `completion_model`and collect the LLM response.

In [ ]:
df = session.sql(f"""
    select *,  SNOWFLAKE.CORTEX.COMPLETE('{completion_model}', prompt, {{'temperature': 0}}) as LLMAnswer 
    from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_targetLLMReadyPrompt
          
""")

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_targetLLMAnswer", mode="overwrite", table_type='transient')
df.limit(5)


# 8. Working with the Data: Factor Construction
## 8.1 Executive On/Off Topic Factor
When an executive answer is semantically similar (dissimilar) to the analyst’s question, it suggests that the answer uses language and concepts similar to (different from) the analyst question, indicating it is on-topic (off-topic). To determine semantic closeness, we vector-embed the question-and-answer texts and calculate a cosine similarity score between the two vectors.

### 8.1.1 Question vs Executive Answer Cosine Similarity

In [ ]:
df = session.sql(f'''

    with vec as(
    select *
        , {snf_embed_text_func}('{embedding_model}', question) as questionVec
        , {snf_embed_text_func}('{embedding_model}',answer) as answerVec 
    from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_targetLLMAnswer
    )
    select *, VECTOR_COSINE_SIMILARITY(questionVec, answerVec) as execOnOffTopicFactor from vec 
               
''')

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_execOnOffTopicFactor", mode="overwrite", table_type='transient')
df.limit(5)


### 8.1.2 Transcript Mean Executive On/Off Topic Factor
Cosine similarity scores are averaged at the transcript level. A high (low) Cosine Similarity Score indicates an On (Off) Topic Executive.



In [ ]:
select avg(execOnOffTopicFactor) as transcriptLevelExecOnOffTopicFactor 
from {{sp_llm_qs_location}}.Russell_Constituents_TranscriptComponents_execOnOffTopicFactor


## 8.2 Executive Proactive/Reactive Factor
### 8.2.1 Question vs LLM Answer Cosine Similarity
Since the LLM answers only within the context of information provided in the prepared remarks, a high (low) cosine similarity score indicates that the LLM answers are semantically similar (dissimilar) the questions, reflecting the executives are proactive (reactive).

In [ ]:
df = session.sql(f'''

    with vec as(
    select *, {snf_embed_text_func}('{embedding_model}', question) as questionVec, 
    {snf_embed_text_func}('{embedding_model}',LLMAnswer['choices'][0]['messages']::string) as LLMAnswerVec 
        from {sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_targetLLMAnswer
    )
    select *, VECTOR_COSINE_SIMILARITY(questionVec, LLMAnswerVec) as execProactiveReactiveFactor from vec 
               
''')

df.write.save_as_table(f"{sp_llm_qs_location}.Russell_Constituents_TranscriptComponents_execProactiveReactiveFactor", mode="overwrite", table_type='transient')
df.limit(5)


### 8.2.2 Transcript Mean Executive Proactive/Reactive Factor
Similar to the construction of the Executive On/Off Topic factor, both the LLM answers and questions are summarized, vector-embedded and cosine similarity scores are averaged at the transcript level.

In [ ]:
select avg(execProactiveReactiveFactor) as transcriptLevelexecProactiveReactiveFactor 
from {{sp_llm_qs_location}}.Russell_Constituents_TranscriptComponents_execProactiveReactiveFactor

# 9. Results & Summary
This research underscores the significant impact of executive communication styles during earnings calls on firm performance. Proactive executives who anticipate market concerns and provide concise, on-topic responses foster transparency, aligning with investor expectations and driving superior returns. The findings demonstrate that firms with Efficient Communicators achieve statistically significant outperformance, while Total Redirectors suffer from diminished confidence and underperformance. These insights validate the critical role of strategic communication in shaping investor perceptions and influencing market outcomes.

Advanced analytical tools, such as vector embeddings and cosine similarity metrics, enable nuanced evaluations of executive-analyst interactions, revealing measurable performance effects across different communication styles. While large language models (LLMs) enhance feature extraction, challenges like forward-looking bias and inconsistency highlight the need for caution in time-sensitive tasks. Overall, the integration of proactive, clear, and relevant communication strategies remains paramount in fostering investor trust and maximizing financial success in a competitive marketplace.